In [1]:
import warnings
warnings.simplefilter('ignore')
import tensorflow as tf

In [2]:
print("Is GPU available?", tf.test.is_gpu_available())
print("TF version:", tf.__version__)
print("Keras version:", tf.keras.__version__)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
Is GPU available? True
TF version: 2.3.1
Keras version: 2.4.0


In [3]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MSE

from collections import deque
import numpy as np
import matplotlib.pyplot as plt
import random
import gym

In [4]:
class Critic(tf.keras.Model):
    def __init__(self):
        super(Critic, self).__init__()
        self.f1 = tf.keras.layers.Dense(512, activation='relu')
        self.f2 = tf.keras.layers.Dense(512, activation='relu')
        self.v =  tf.keras.layers.Dense(1, activation=None)

    def call(self, inputstate, action):
        x = self.f1(tf.concat([inputstate, action], axis=1))
        x = self.f2(x)
        x = self.v(x)
        return x


class Actor(tf.keras.Model):
    def __init__(self, no_action):
        super(Actor, self).__init__()    
        self.f1 = tf.keras.layers.Dense(512, activation='relu')
        self.f2 = tf.keras.layers.Dense(512, activation='relu')
        self.mu =  tf.keras.layers.Dense(no_action, activation='tanh')

    def call(self, state):
        x = self.f1(state)
        x = self.f2(x)
        x = self.mu(x)  
        return x

In [5]:
class Agent():
    def __init__(self, n_actions, min_action, max_action):
        self.actor_main = Actor(n_actions)
        self.actor_target = Actor(n_actions)
        self.critic_main = Critic()
        self.critic_main2 = Critic()
        self.critic_target = Critic()
        self.critic_target2 = Critic()
        self.batch_size = 64
        self.n_actions = n_actions
        self.a_opt = tf.keras.optimizers.Adam(0.001)
        self.c_opt1 = tf.keras.optimizers.Adam(0.002)
        self.c_opt2 = tf.keras.optimizers.Adam(0.002)
        self.memory = deque(maxlen=100000)
        self.trainstep = 0
        self.gamma = 0.99
        self.min_action = min_action
        self.max_action = max_action
        self.actor_update_steps = 2
        self.warmup = 200
        self.actor_target.compile(optimizer=self.a_opt)
        self.critic_target.compile(optimizer=self.c_opt1)
        self.critic_target2.compile(optimizer=self.c_opt2)
        self.tau = 0.005


    def act(self, state, evaluate=False):
        if self.trainstep > self.warmup:
            evaluate = True
        state = tf.convert_to_tensor([state], dtype=tf.float32)
        actions = self.actor_main(state)
        if not evaluate:
            actions += tf.random.normal(shape=[self.n_actions], mean=0.0, stddev=0.1)

        actions = self.max_action * (tf.clip_by_value(actions, self.min_action, self.max_action))
        
        return actions[0]
    
    def store(self, state, action, reward, n_state, done):
        pack = [state, action, reward, n_state, 1 - int(done)]
        self.memory.append(pack)
    
    def take_data(self, batch_size):
        pack = random.sample(self.memory, batch_size)
        states = []
        actions = []
        rewards = []
        n_states = []
        dones = []
        for i in range(batch_size):
            states.append(pack[i][0])
            actions.append(pack[i][1])
            rewards.append(pack[i][2])
            n_states.append(pack[i][3])
            dones.append(pack[i][4])
        return states, actions, rewards, n_states, dones

    def update_target(self, tau=None):

        if tau is None:
            tau = self.tau

        weights1 = []
        targets1 = self.actor_target.weights
        for i, weight in enumerate(self.actor_main.weights):
            weights1.append(weight * tau + targets1[i]*(1-tau))
        self.actor_target.set_weights(weights1)

        weights2 = []
        targets2 = self.critic_target.weights
        for i, weight in enumerate(self.critic_main.weights):
            weights2.append(weight * tau + targets2[i]*(1-tau))
        self.critic_target.set_weights(weights2)


        weights3 = []
        targets3 = self.critic_target2.weights
        for i, weight in enumerate(self.critic_main2.weights):
            weights3.append(weight * tau + targets3[i]*(1-tau))
        self.critic_target2.set_weights(weights3)
    
    def upgrade(self):
        if len(self.memory) < self.batch_size:
            return 

        states, actions, rewards, n_states, dones = self.take_data(self.batch_size)

        states = tf.convert_to_tensor(states, dtype= tf.float32)
        n_states = tf.convert_to_tensor(n_states, dtype= tf.float32)
        rewards = tf.convert_to_tensor(rewards, dtype= tf.float32)
        actions = tf.convert_to_tensor(actions, dtype= tf.float32)

        with tf.GradientTape() as tape1, tf.GradientTape() as tape2:

            target_actions = self.actor_target(n_states)
            target_actions += tf.clip_by_value(tf.random.normal(shape=[*np.shape(target_actions)], mean=0.0, stddev=0.2), -0.5, 0.5)
            target_actions = self.max_action * (tf.clip_by_value(target_actions, self.min_action, self.max_action))


            target_next_state_values = tf.squeeze(self.critic_target(n_states, target_actions), 1)
            target_next_state_values2 = tf.squeeze(self.critic_target2(n_states, target_actions), 1)

            critic_value = tf.squeeze(self.critic_main(states, actions), 1)
            critic_value2 = tf.squeeze(self.critic_main2(states, actions), 1)

            next_state_target_value = tf.math.minimum(target_next_state_values, target_next_state_values2)

            target_values = rewards + self.gamma * next_state_target_value * dones
            critic_loss1 = tf.keras.losses.MSE(target_values, critic_value)
            critic_loss2 = tf.keras.losses.MSE(target_values, critic_value2)




        grads1 = tape1.gradient(critic_loss1, self.critic_main.trainable_variables)
        grads2 = tape2.gradient(critic_loss2, self.critic_main2.trainable_variables)

        self.c_opt1.apply_gradients(zip(grads1, self.critic_main.trainable_variables))
        self.c_opt2.apply_gradients(zip(grads2, self.critic_main2.trainable_variables))


        self.trainstep +=1

        if self.trainstep % self.actor_update_steps == 0:

            with tf.GradientTape() as tape3:

                new_policy_actions = self.actor_main(states)
                actor_loss = -self.critic_main(states, new_policy_actions)
                actor_loss = tf.math.reduce_mean(actor_loss)

            grads3 = tape3.gradient(actor_loss, self.actor_main.trainable_variables)
            self.a_opt.apply_gradients(zip(grads3, self.actor_main.trainable_variables))

        self.update_target()

In [6]:
env = gym.make('Pendulum-v0')

print('Min_action:', env.action_space.low[0])
print('Max_action: ', env.action_space.high[0])
print('States: ', env.observation_space.shape)
print('Actions: ', env.action_space.shape)

Min_action: -2.0
Max_action:  2.0
States:  (3,)
Actions:  (1,)


In [ ]:
agent = Agent(1, -2, 2)

n_episodes = 450
avg_hist = []
scores = []

for i in range(n_episodes):
    score = 0 
    state = env.reset()
    done = False

    while not done:
        action = agent.act(state)
        n_state, reward, done, _ = env.step(action)
        agent.store(state, action, reward, n_state, done)
        agent.upgrade()
        state = n_state
        score += reward
    
    scores.append(score)
    avg_reward = int(np.mean(scores[-100:]))
    avg_hist.append(avg_reward)
    print(f'Episode: {i}  Score: {score}  AVG: {avg_reward}')

env.close()

Episode: 0  Score: -1251.8430744505442  AVG: -1251
Episode: 1  Score: -1379.0881423055941  AVG: -1315
Episode: 2  Score: -1251.5184371825876  AVG: -1294
Episode: 3  Score: -1500.7020938201556  AVG: -1345
Episode: 4  Score: -1373.9635071934324  AVG: -1351
Episode: 5  Score: -1316.2674362617392  AVG: -1345
Episode: 6  Score: -1493.9209945869143  AVG: -1366
Episode: 7  Score: -1145.9944346971308  AVG: -1339
Episode: 8  Score: -650.9026322807301  AVG: -1262
Episode: 9  Score: -141.32949129832198  AVG: -1150
Episode: 10  Score: -1111.8862754014403  AVG: -1147
Episode: 11  Score: -817.4669366343174  AVG: -1119
Episode: 12  Score: -138.37904786096271  AVG: -1044
Episode: 13  Score: -4.4492705557682495  AVG: -969
Episode: 14  Score: -1088.341537807602  AVG: -977
Episode: 15  Score: -1.9713717175413945  AVG: -916
Episode: 16  Score: -405.6409349009683  AVG: -886
Episode: 17  Score: -1572.6064525717309  AVG: -924
Episode: 18  Score: -424.099635819068  AVG: -898
Episode: 19  Score: -1536.04338007

Episode: 161  Score: -319.6733139605201  AVG: -145
Episode: 162  Score: -2.305970439590636  AVG: -144
Episode: 163  Score: -227.62173122222063  AVG: -145
Episode: 164  Score: -124.70371367779273  AVG: -145
Episode: 165  Score: -6.399828987152788  AVG: -144
Episode: 166  Score: -232.51331272952055  AVG: -144
Episode: 167  Score: -266.78083658009996  AVG: -147
Episode: 168  Score: -328.2548100470968  AVG: -149
Episode: 169  Score: -132.31860354818662  AVG: -149
Episode: 170  Score: -316.0283939565849  AVG: -150
Episode: 171  Score: -120.29627293901221  AVG: -151
Episode: 172  Score: -118.38370316410224  AVG: -151
Episode: 173  Score: -122.73015940161335  AVG: -151
Episode: 174  Score: -117.25315500935292  AVG: -151
Episode: 175  Score: -248.3898733314541  AVG: -152
Episode: 176  Score: -122.02073942740392  AVG: -152
Episode: 177  Score: -122.115865315726  AVG: -152
Episode: 178  Score: -231.76005118663645  AVG: -154
Episode: 179  Score: -1.6333105342124505  AVG: -153
Episode: 180  Score:

Episode: 320  Score: -115.79097384256832  AVG: -141
Episode: 321  Score: -226.4726397060023  AVG: -141
Episode: 322  Score: -122.50664475218056  AVG: -140
Episode: 323  Score: -233.53551198368365  AVG: -142
Episode: 324  Score: -117.6484934987224  AVG: -143
Episode: 325  Score: -238.9666493182939  AVG: -144
Episode: 326  Score: -116.3187060517222  AVG: -144
Episode: 327  Score: -331.64207530517956  AVG: -146
Episode: 328  Score: -233.2206312989046  AVG: -147
Episode: 329  Score: -121.35792054719158  AVG: -147
Episode: 330  Score: -1.577401356918449  AVG: -146
Episode: 331  Score: -119.13701023693908  AVG: -146
Episode: 332  Score: -120.32321272199414  AVG: -146
Episode: 333  Score: -2.388240454236846  AVG: -146
Episode: 334  Score: -119.63056161350259  AVG: -146
Episode: 335  Score: -126.24729463236908  AVG: -145
Episode: 336  Score: -253.8798298093393  AVG: -144
Episode: 337  Score: -116.63295717335805  AVG: -144
Episode: 338  Score: -237.93263564667313  AVG: -145
Episode: 339  Score:

In [ ]:
plt.plot(avg_hist)
plt.grid()
plt.xlabel('Episodes')
plt.ylabel('Avg rewards')

In [ ]:
env = gym.make('Pendulum-v0')

for i in range(10):
    state = env.reset()
    done = False
    
    while not done:
        action = agent.act(state, True)
        n_state, _, done, _ = env.step(action)
        state = n_state
        env.render()
env.close()

In [ ]:
agent.actor_main.save_weights('actor_weights.h5')